## Example Notebook: Using VideoAgent

### 🛠️ Setup Instructions

Before running this notebook:

- Ensure you have created an `.env` file in the **same directory** as this notebook. It must contain all required environment variables (e.g., API keys or configuration values expected by `VideoAgent`).
- Make sure all required libraries are installed by running:

  ```bash
  pip install -r requirements.txt
  ```

### About

**VideoAgent** operates in two key stages:

1. **Video Retrieval** – Given a user query, the agent retrieves relevant videos from a pre-ingested **Azure AI Search index**. This search ensures that only contextually relevant videos are passed on for deep analysis.

2. **Video Question Answering (QA)** – After retrieval, the agent uses the **Multi-Modal Critical Thinking (MMCT)** framework ([arxiv.org/abs/2405.18358](https://arxiv.org/abs/2405.18358)) to generate a high-quality answer. MMCT involves two agents:

   - **Planner**: Drives the reasoning process using a structured toolchain, generating an initial response.
   - **Critic (optional)**: Analyzes the planner’s output and, if needed, provides feedback that prompts an improved final answer.

> **Note:** The critic agent is enabled by default. You can disable it by setting `use_critic_agent=False` during initialization.  
> **Disabling the critic agent skips the feedback loop and may reduce the accuracy of the final response.**

---

### Tool Workflow

Unlike independent tool selection, **VideoAgent uses a fixed pipeline** of tools that work collaboratively during the QA stage. These tools are automatically orchestrated by the planner:

- `GET_SUMMARY_TRANSCRIPT` – Extracts the full transcript and a high-level visual summary of the video.
- `QUERY_SUMMARY_TRANSCRIPT` – Given a query, this tool identifies **three timestamps** in the transcript that are most relevant.
- `QUERY_AZURE_COMPUTER_VISION` _(optional)_ – Uses **Azure Computer Vision** to return **three additional timestamps** related to the visual content of the query.
- `QUERY_GPT4V` – Uses **OpenAI's GPT-4V** to inspect video frames around the identified timestamps and generate a detailed response grounded in both visual and textual understanding.

By default, all tools are used in a coordinated pipeline. You can disable **only** the Azure Computer Vision tool by setting `use_azure_cv_tool=False` during agent initialization.


### Importing Libaries


In [ ]:
import asyncio
import ast
from mmct.video_pipeline import VideoAgent

# nest_asyncio is used to allow nested event loops in Jupyter notebooks
import nest_asyncio


nest_asyncio.apply()

import warnings

warnings.filterwarnings("ignore")

In [ ]:
# What are the steps involved in applying Beejamrutham to seeds?  -  Answer in the transcript file itself
# What local ingredients are used in the preparation of Beejamrutham?

In [ ]:
# This code is used to run the VideoAgent class
query = "What local ingredients are used in the preparation of Beejamrutham?"
index_name = "telangana-video-index-latest-2"  # Azure AI Search index name
top_n = 2  # Number of top results (video ids for MMCT VQnA) to return from the index
use_azure_cv_tool = False
use_critic_agent = True
stream = True

video_agent = VideoAgent(
    query=query,
    index_name=index_name,
    top_n=top_n,
    use_azure_cv_tool=use_azure_cv_tool,
    use_critic_agent=use_critic_agent,
    stream=stream,
)

response = asyncio.run(video_agent())
print(response)